In [ ]:
import arcade
import random
import math

SCREEN_WIDTH, SCREEN_HEIGHT = 800, 400
TITLE = "Multi-Ball Bounce with Gravity & Collisions"

BALL_COUNT = 20
GRAVITY = -900
REST = 0.85  # restitution (energy kept on bounce)


class Ball(arcade.Sprite):
    def __init__(self, x, y):
        super().__init__(":resources:images/items/coinGold.png", scale=0.6)
        self.center_x = x
        self.center_y = y
        self.vx = random.uniform(-250, 250)
        self.vy = random.uniform(-100, 200)
        self.mass = 1.0 


class BounceDemo(arcade.Window):
    def __init__(self):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, TITLE)
        arcade.set_background_color(arcade.color.SKY_BLUE)

        self.ball_list = arcade.SpriteList()
        self.ground_h = 60
        self.ground_rect = arcade.rect.LBWH(0, 0, SCREEN_WIDTH, self.ground_h)

        self.reset_game()

    # create/refresh the balls
    def reset_game(self):
        self.ball_list = arcade.SpriteList()
        for _ in range(BALL_COUNT):
            x = random.randint(100, SCREEN_WIDTH - 100)
            y = random.randint(300, SCREEN_HEIGHT - 50)
            ball = Ball(x, y)
            self.ball_list.append(ball)

    def on_draw(self):
        self.clear()
        arcade.draw_rect_filled(self.ground_rect, arcade.color.DARK_SPRING_GREEN)
        self.ball_list.draw()

        # Optional UI hint
        arcade.draw_text(
            "Press R to restart",
            10, SCREEN_HEIGHT - 30,
            arcade.color.BLACK, 14
        )

    def physics_step(self, dt):
        for ball in self.ball_list:
            # Apply gravity
            ball.vy += GRAVITY * dt

            # Move
            ball.center_x += ball.vx * dt
            ball.center_y += ball.vy * dt

            # Floor bounce
            floor_y = self.ground_h + ball.height / 2
            if ball.center_y < floor_y:
                ball.center_y = floor_y
                ball.vy = -ball.vy * REST

            # Wall bounces
            left = ball.width / 2
            right = SCREEN_WIDTH - ball.width / 2
            if ball.center_x < left or ball.center_x > right:
                ball.vx = -ball.vx * REST
                ball.center_x = max(left, min(ball.center_x, right))

            # Ceiling
            top = SCREEN_HEIGHT - ball.height / 2
            if ball.center_y > top:
                ball.center_y = top
                ball.vy = -ball.vy * REST

    def resolve_collisions(self):
        # Pairwise collisions
        for i, a in enumerate(self.ball_list):
            for b in self.ball_list[i + 1:]:
                dx = b.center_x - a.center_x
                dy = b.center_y - a.center_y
                dist = math.hypot(dx, dy)
                min_dist = (a.width + b.width) / 2 * 0.5 # small overlap tolerance

                if dist < min_dist and dist > 0:

                    # Normal vector
                    nx, ny = dx / dist, dy / dist

                    # Relative velocity
                    rvx = b.vx - a.vx
                    rvy = b.vy - a.vy

                    # Relative speed along normal
                    vel_norm = rvx * nx + rvy * ny
                    if vel_norm > 0:
                        continue

                    impulse = -(1 + REST) * vel_norm
                    impulse = (1/a.mass + 1/b.mass)

                    # Apply impulses
                    a.vx -= (impulse / a.mass) * nx
                    a.vy -= (impulse / a.mass) * ny
                    b.vx += (impulse / b.mass) * nx
                    b.vy += (impulse / b.mass) * ny

                    # Push the balls apart slightly
                    overlap = min_dist - dist
                    a.center_x -= nx * overlap / 2
                    a.center_y -= ny * overlap / 2
                    b.center_x += nx * overlap / 2
                    b.center_y += ny * overlap / 2

    def on_update(self, dt):
        self.physics_step(dt)
        self.resolve_collisions()

    def on_key_press(self, symbol, modifiers):
        if symbol == arcade.key.R:
            self.reset_game()

window = BounceDemo()
arcade.run()

In [ ]:
#tinyurl.com/sceneees

import arcade
import random
import math

SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
TITLE = "Multi-Ball Bounce"

BALL_COUNT = 10
GRAVITY = -900
REST = 1.01

# Explosion tuning
EXPLOSION_RADIUS = 180
EXPLOSION_STRENGTH = 2500  # try 1500–4000

TEXTURE_PATHS = [
    ":resources:images/items/coinGold.png",
    ":resources:images/items/gemBlue.png",
    ":resources:images/items/gemGreen.png",
    ":resources:images/items/gemRed.png",
]

class Ball(arcade.Sprite):
    def __init__(self, x, y):
        # pick a random coin/gem
        texture_path = random.choice(TEXTURE_PATHS)
        super().__init__(texture_path, scale=0.6)
        self.center_x = x
        self.center_y = y
        self.vx = random.uniform(-250, 250)
        self.vy = random.uniform(-100, 200)
        self.mass = 1.0